# Features engineering

In [8]:
# Libraries and parameters
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import math

In [9]:
# Import data
data = pd.read_csv("../raw_data/final_raw_data.csv", index_col="Unnamed: 0")

In [10]:
# #Creating hitter stats up to at bat
data['hitter_previous_stats_szn'] = data.groupby("hitter_id")["y_target"].cumsum() / (data.groupby('hitter_id').cumcount() + 1)
data['rolling_1ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

In [11]:
# Calculate stats of pitcher before he pitchs
data['pitcher_previous_stats_szn'] = data.groupby("pitcher_id")["y_target"].cumsum() / (data.groupby('pitcher_id').cumcount() + 1)
data['rolling_1pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

In [13]:
data.head()

,id,game_id,hitter_id,hitter_hand,pitcher_id,pitcher_hand,temp_f,humidity,at_bat_end_time,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,play_outcome,y_target,pitch_class,day_night,home_team_id,away_team_id,attendance,venue_id,hitter_player_name,hitter_position,hitter_primary_position,pitcher_player_name,pitcher_primary_position,hitter_previous_stats_szn,rolling_1ab,rolling_3ab,rolling_10ab,pitcher_previous_stats_szn,rolling_1pitch,rolling_3pitch,rolling_10pitch,handed_matchup
0,d43f7325-ca19-49ab-94f6-b92055c26dd5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,041632a9-afb2-4ec3-b1de-9b0bbe33ab64,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:11:07+00:00,94.4,4.0,0.0,0.0,walk,1,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Wade Jr., LaMonte",IF,1B,"Cole, Gerrit",SP,1.0,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,LR
1,f7f6d34f-5e1e-4ca9-896a-1f026054cbf5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,e5bdeb0e-38fc-4d30-8127-43d0d5b2864d,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:13:06+00:00,86.8,5.0,4.0,0.0,SO,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Conforto, Michael",OF,RF,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.500000,1.0,1.000000,1.000000,LR
2,faad7e49-b9a2-4359-85a4-695438b6885c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,45bca5ce-4514-4a91-9410-e201c7b0052d,R,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:14:55+00:00,89.8,5.0,9.0,1.0,SO,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Flores, Wilmer",IF,1B,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.333333,0.0,0.500000,0.500000,RR
3,8f33e404-d2c3-427e-8a98-96ea1945285e,f3f0ae8e-cb65-4e96-9530-c6f868738f09,43a9d631-5673-4059-9b25-d59290bc32c3,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,32.0,2023-03-30 17:18:09+00:00,89.1,7.0,14.0,2.0,SO,0,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Pederson, Joc",OF,LF,"Cole, Gerrit",SP,0.0,NaN,NaN,NaN,0.250000,0.0,0.333333,0.333333,LR
4,b2fbcf1d-862e-4102-bb4e-fbac3656031c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,d1c07b01-a86d-4c17-ac38-8217e364c2cf,R,72248e3e-3b0f-4523-9ff6-833aa33f3b32,R,38.0,32.0,2023-03-30 17:21:39+00:00,85.9,4.0,0.0,0.0,SO,0,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"LeMahieu, David",IF,3B,"Webb, Logan",SP,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,RR


In [19]:
#Create handed matchups

data["handed_matchup"] = data["hitter_hand"] + data["pitcher_hand"]
data["handed_matchup"] = data.handed_matchup.apply((lambda x: 0 if x[0] == x[1] else 1))
data = data.drop(columns=["hitter_hand", "pitcher_hand"])

In [20]:
#total hitter and pitcher at bats prior to at bat
data["hitter_ab_count"] = data.groupby('hitter_id').cumcount()
data["pitcher_ab_count"] = data.groupby('pitcher_id').cumcount()
data["match_up_ab_count_delta"] = data["pitcher_ab_count"] - data["hitter_ab_count"] 

In [22]:
#Balls and strikes efficiencies, batters
data["reverse_pitch_class"] = data["pitch_class"].apply(lambda x: 0 if x == 1 else 1)

data["hitter_strikes_atb"] = (data.groupby("hitter_id")["reverse_pitch_class"].cumsum())
data["hitter_balls_atb"] = (data.groupby("hitter_id")["pitch_class"].cumsum())

data["hitter_strikes_only_results"] = data["y_target"] * data["reverse_pitch_class"]
data["hitter_balls_only_results"] = data["y_target"] * data["pitch_class"]

data["hitter_strikes_results"] = (data.groupby("hitter_id")["hitter_strikes_only_results"].cumsum())
data["hitter_balls_results"] = (data.groupby("hitter_id")["hitter_balls_only_results"].cumsum())

data["hitter_strikes_eff"] = data["hitter_strikes_results"] / data["hitter_strikes_atb"]
data["hitter_balls_eff"] = data["hitter_balls_results"] / data["hitter_balls_atb"]

#Balls and strikes efficiencies, pitchers

data["pitcher_total_atb"] = (data.groupby("pitcher_id").cumcount())
data["pitcher_strikes_spread"] = (data.groupby("pitcher_id")["reverse_pitch_class"].cumsum()) / data["pitcher_total_atb"]
data["pitcher_balls_spread"] = (data.groupby("pitcher_id")["pitch_class"].cumsum()) / data["pitcher_total_atb"]


#pitch speed stats
data["hitter_speeds_class1"] = data["pitch_speed_mph"] * data["y_target"]
data["hitter_success_speed"] = (data.groupby("hitter_id")["hitter_speeds_class1"].cumsum()) / (data.groupby("hitter_id")["y_target"].cumsum())

data["pitcher_speed"] = (data.groupby("pitcher_id")["pitch_speed_mph"].cumsum())/data["pitcher_total_atb"]


#dropping redundant columns to avoid multi-colinearity

data = data.drop(columns=["pitch_class", "reverse_pitch_class", "hitter_strikes_atb", "hitter_balls_atb", "hitter_strikes_only_results",
                         "hitter_balls_only_results", "hitter_strikes_results", "hitter_balls_results", "pitcher_total_atb", "hitter_speeds_class1"])

In [23]:
#drop 0.004343% of data (first ten at bats for each player)
data = data.dropna(subset=["rolling_10ab", "rolling_10pitch", "hitter_strikes_eff", "hitter_balls_eff", "hitter_success_speed"])

In [24]:
data.isnull().sum()

id                                    0
game_id                               0
hitter_id                             0
pitcher_id                            0
temp_f                                0
humidity                              0
at_bat_end_time                       0
pitch_speed_mph                       0
pitch_count_at_bat                    0
pitcher_pitch_count_at_bat_start      0
outs_at_start                         0
play_outcome                          0
y_target                              0
day_night                             0
home_team_id                          0
away_team_id                          0
attendance                          438
venue_id                              0
hitter_player_name                    0
hitter_position                       0
hitter_primary_position               0
pitcher_player_name                   0
pitcher_primary_position              0
hitter_previous_stats_szn             0
rolling_1ab                           0


In [25]:
data.columns

Index(['id', 'game_id', 'hitter_id', 'pitcher_id', 'temp_f', 'humidity',
       'at_bat_end_time', 'pitch_speed_mph', 'pitch_count_at_bat',
       'pitcher_pitch_count_at_bat_start', 'outs_at_start', 'play_outcome',
       'y_target', 'day_night', 'home_team_id', 'away_team_id', 'attendance',
       'venue_id', 'hitter_player_name', 'hitter_position',
       'hitter_primary_position', 'pitcher_player_name',
       'pitcher_primary_position', 'hitter_previous_stats_szn', 'rolling_1ab',
       'rolling_3ab', 'rolling_10ab', 'pitcher_previous_stats_szn',
       'rolling_1pitch', 'rolling_3pitch', 'rolling_10pitch', 'handed_matchup',
       'hitter_ab_count', 'pitcher_ab_count', 'match_up_ab_count_delta',
       'hitter_strikes_eff', 'hitter_balls_eff', 'pitcher_strikes_spread',
       'pitcher_balls_spread', 'hitter_success_speed', 'pitcher_speed'],
      dtype='object')

In [26]:
#Creating final training set with only ket features
modeling_columns = ['temp_f', 'humidity', 'pitcher_pitch_count_at_bat_start', 'outs_at_start','hitter_position',
                    'hitter_previous_stats_szn', 'rolling_1ab','rolling_3ab', 'rolling_10ab', 
                    'pitcher_previous_stats_szn','rolling_1pitch', 'rolling_3pitch', 'rolling_10pitch', 
                    'handed_matchup', 'match_up_ab_count_delta',
                    'hitter_strikes_eff', 'hitter_balls_eff', 'pitcher_strikes_spread','pitcher_balls_spread', 
                    'hitter_success_speed', 'pitcher_speed', 'y_target']

data = data[modeling_columns]
data

,temp_f,humidity,pitcher_pitch_count_at_bat_start,outs_at_start,hitter_position,hitter_previous_stats_szn,rolling_1ab,rolling_3ab,rolling_10ab,pitcher_previous_stats_szn,rolling_1pitch,rolling_3pitch,rolling_10pitch,handed_matchup,match_up_ab_count_delta,hitter_strikes_eff,hitter_balls_eff,pitcher_strikes_spread,pitcher_balls_spread,hitter_success_speed,pitcher_speed,y_target
20,40.0,30.0,44.0,1.0,IF,0.500000,1.0,1.000000,1.0,0.200000,0.0,0.000000,0.222222,1,8,0.000000,1.000000,0.666667,0.444444,94.400000,103.688889,0
32,40.0,30.0,45.0,2.0,IF,0.500000,1.0,1.000000,1.0,0.250000,0.0,0.333333,0.300000,1,10,0.000000,1.000000,0.727273,0.363636,87.700000,98.254545,0
65,42.0,26.0,80.0,2.0,IF,0.333333,0.0,0.500000,0.5,0.210526,1.0,0.333333,0.200000,1,16,0.000000,1.000000,0.833333,0.222222,94.400000,99.527778,0
68,42.0,26.0,65.0,0.0,IF,0.500000,1.0,1.000000,1.0,0.277778,0.0,0.333333,0.300000,0,16,0.000000,1.000000,0.647059,0.411765,92.200000,93.858824,0
72,42.0,26.0,70.0,2.0,OF,0.333333,0.0,0.500000,0.5,0.250000,0.0,0.000000,0.200000,0,17,0.500000,0.000000,0.631579,0.421053,92.800000,93.131579,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142302,96.0,28.0,6.0,1.0,IF,0.402145,1.0,0.333333,0.3,0.277228,0.0,0.000000,0.300000,1,-171,0.382900,0.451923,0.666667,0.338308,87.950667,88.826368,1
142303,96.0,28.0,12.0,1.0,IF,0.366548,1.0,0.666667,0.3,0.280788,1.0,0.333333,0.300000,1,-359,0.332487,0.446429,0.668317,0.336634,89.513592,88.842574,1
142304,96.0,28.0,18.0,1.0,OF,0.329545,0.0,0.000000,0.1,0.284314,1.0,0.666667,0.300000,0,-324,0.284830,0.400000,0.669951,0.334975,89.332759,88.860099,1
142305,96.0,28.0,20.0,1.0,C,0.355769,0.0,0.333333,0.2,0.282927,1.0,1.000000,0.400000,0,-3,0.354610,0.358209,0.666667,0.338235,89.832432,88.841176,0


In [27]:
data.to_csv("../preproc_data/data_with_new_features.csv")